In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from utils import *
import tensorflow as tf






In [2]:
def convert_numbers_to_inputs(size_out, numbers):
    converted = [0]*size_out
    for _idx in numbers:
        if _idx<len(converted):
            converted[_idx] = 1
    
    return converted

def convert_inputs_to_numbers(vector):
    numbers = []

    for vi in range(len(vector)):
        if vector[vi]>=0.99:
            numbers.append(vi)
    return numbers

def load_inputs_and_outputs(input_size, output_size, used_numbers_count):
    db = load_db(os.path.join("database", "lottoDB"))
    tkeys = list(db.keys())

    idx = 0
    x = []
    y = []

    db_numbers = []
    while idx < (len(tkeys)-1):
        _i = db[tkeys[idx]][0][0:used_numbers_count]
        _o = db[tkeys[idx + 1]][0][0:used_numbers_count]

        _ci = convert_numbers_to_inputs(input_size, _i)
        _co = convert_numbers_to_inputs(output_size, _o)

        x.append(_ci)
        y.append(_co)

        db_numbers.append(_i)
        idx += 1
    return x , y, db_numbers

# load inputs
LOTTO_NUMBER_MAX = 50 
USED_NUMBERS_COUNT = 5
SIZE_IN = LOTTO_NUMBER_MAX + 1
SIZE_OUT = LOTTO_NUMBER_MAX + 1

#
X, Y, DB = load_inputs_and_outputs(SIZE_IN, SIZE_OUT, USED_NUMBERS_COUNT)


In [3]:
'''Create model
'''

# Define input and output shapes
input_shape = (SIZE_IN,)
output_shape = (SIZE_OUT,)
model = Sequential()

# in
model.add(Dense(50, activation='sigmoid', input_shape=input_shape))

# Add a Dense layer with x units and ReLU activation function
model.add(Dense(50, activation='sigmoid'))

# Add a Dense layer with x units and ReLU activation function
model.add(Dense(50, activation='sigmoid'))


# Add a Dense layer with x units and ReLU activation function
model.add(Dense(50, activation='sigmoid'))

# out
model.add(Dense(SIZE_OUT, activation='relu'))

# compile
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [4]:

# Load values
ITERATIONS_COUNT = 1000
history = model.fit(X, Y, epochs=ITERATIONS_COUNT, batch_size=10,  verbose=2)



Epoch 1/1000
65/65 - 1s - loss: 0.1165 - accuracy: 0.0093 - 877ms/epoch - 13ms/step
Epoch 2/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0295 - 73ms/epoch - 1ms/step
Epoch 3/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0047 - 68ms/epoch - 1ms/step
Epoch 4/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0109 - 68ms/epoch - 1ms/step
Epoch 5/1000
65/65 - 0s - loss: 0.0938 - accuracy: 0.0186 - 70ms/epoch - 1ms/step
Epoch 6/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0109 - 72ms/epoch - 1ms/step
Epoch 7/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0280 - 72ms/epoch - 1ms/step
Epoch 8/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0031 - 81ms/epoch - 1ms/step
Epoch 9/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0233 - 67ms/epoch - 1ms/step
Epoch 10/1000
65/65 - 0s - loss: 0.0937 - accuracy: 0.0047 - 70ms/epoch - 1ms/step
Epoch 11/1000
65/65 - 0s - loss: 0.0938 - accuracy: 0.0124 - 69ms/epoch - 1ms/step
Epoch 12/1000
65/65 - 0s - loss: 0.0936 - accuracy: 0.0078 - 70ms/epoch - 1ms/step
Epoch 13/10

In [5]:
'''Predict next sequence based on input sequence
'''




# predict all
predictions = model.predict(X)

# print predictions for len-1
with open("predictions.txt", "w") as f:
    for i in range(len(Y)):
        p, x, y, db_in = predictions[i], X[i], Y[i], DB[i]

        if i + 1 < len(DB):
            db_next = DB[i+1]

            # Write to file
            f.write(f"#{i} {db_in} -> {db_next}\n")
            for idx__ in range(len(y)):
                if not (p[idx__]<0.01 and y[idx__] < 0.01):
                    f.write("(%d)%f %f \n"%(idx__, p[idx__],y[idx__]))


# stats
loss, accuracy = model.evaluate(X, Y)
#print(f"loss={loss:4.4f} accuracy={100*accuracy:4.4f}%")

# predict for most recent Y
indices = [-1]
predictions = model.predict([Y[i] for i in indices])
for i in indices:
    p =  predictions[i]

    log(f"# Predicted for {convert_inputs_to_numbers(Y[i])}")
    for idx__ in range(len(y)):
        if p[idx__] > 0.01:
            log(f"({idx__}){ p[idx__]:4.4f}\n")

1/1 [==============================] - 0s 46ms/step
20:19:41.473319 ## # Predicted for [1, 3, 29, 45, 47]
20:19:41.473319 ## (1)1.1694

20:19:41.473319 ## (23)0.6478

20:19:41.473319 ## (29)1.0362

20:19:41.473319 ## (47)0.3231

20:19:41.473319 ## (50)1.5304

